# Librerias

In [331]:
import pandas as pd
import numpy as np
import time
import sys

# Lectura de Datos

In [332]:
archivo=pd.read_csv('Parallel_Machine_instances/Instance 1.txt', header=None, delim_whitespace=True)
tamaño_archivo=len(archivo)
numero_columnas=len(archivo.columns)

In [333]:
numero_tareas=int(archivo.iloc[-2][0])
numero_maquinas=int(archivo.iloc[-2][1])
matriz_tiempos=[]
for i in range(numero_tareas):
    j=1
    while j<(numero_columnas):
        matriz_tiempos.append(archivo.iloc[i][j])
        j+=2
    
matriz_tiempos = np.asarray(matriz_tiempos)
matriz_tiempos= matriz_tiempos.astype(int)
matriz_tiempos=matriz_tiempos.reshape(numero_tareas,numero_maquinas)


# Solución inicial Greedy

A cada tarea se asigna a la maquina con menor tiempo de ejecución

In [334]:
asignacion_maquinas={}
tiempo_maquina={}
for i in range(numero_maquinas):
    asignacion_maquinas[i]=[]

for i in range(numero_maquinas):
    tiempo_maquina[i]=[]
 
for i in range(numero_tareas):
    maquina_asignar=np.argmin(matriz_tiempos[i])
    tiempo_maquina[maquina_asignar].append(np.min(matriz_tiempos[i]))
    asignacion_maquinas[maquina_asignar].append(i)
print("Tareas asignadas a maquinas= ",asignacion_maquinas)
print("tiempo de uso cada maquina= ",tiempo_maquina)

Tareas asignadas a maquinas=  {0: [12, 56, 60, 69, 77, 89, 99], 1: [5, 6, 11, 13, 24, 26, 42, 68, 83, 87], 2: [4, 7, 16, 35, 53, 63, 65, 66, 78, 80], 3: [2, 18, 20, 25, 34, 36, 41, 46, 51, 59, 75, 81, 84], 4: [3, 19, 27, 28, 33, 37, 40, 45, 61, 71, 76, 90, 93, 96], 5: [49, 52, 54, 58, 67, 82, 85, 94], 6: [0, 29, 48, 55, 57, 70, 72, 79, 91], 7: [31, 32, 43, 50, 74, 86, 98], 8: [1, 9, 10, 15, 21, 30, 38, 39, 44, 47, 62, 92, 95], 9: [8, 14, 17, 22, 23, 64, 73, 88, 97]}
tiempo de uso cada maquina=  {0: [1, 17, 13, 7, 4, 12, 3], 1: [21, 16, 5, 4, 4, 13, 8, 10, 17, 1], 2: [12, 33, 2, 7, 23, 13, 10, 4, 2, 30], 3: [14, 24, 20, 3, 13, 16, 30, 6, 18, 21, 14, 6, 29], 4: [33, 1, 7, 8, 12, 6, 11, 14, 10, 8, 5, 25, 2, 4], 5: [10, 2, 3, 4, 1, 37, 9, 10], 6: [4, 5, 16, 1, 9, 1, 11, 15, 20], 7: [2, 12, 2, 5, 4, 4, 9], 8: [7, 5, 5, 15, 6, 30, 2, 2, 8, 5, 26, 1, 51], 9: [1, 9, 9, 13, 14, 21, 8, 16, 5]}


# Funcion Fitness

In [335]:
def evaluacion(numero_maquinas,tiempo_maquina):
    tiempos_final_maquina=[]
    for i in range(numero_maquinas):
        tiempos_final_maquina.append(int(sum(tiempo_maquina[i])))    
    makespan=max(tiempos_final_maquina)
    return(makespan,tiempos_final_maquina)


In [336]:
makespan,tiempos_final_maquina=evaluacion(numero_maquinas,tiempo_maquina)
print(makespan)
print(tiempos_final_maquina)

214
[57, 99, 136, 214, 146, 76, 82, 38, 163, 96]


# Funcion busca maquinas por tarea

In [337]:
def maquina_por_tarea(tarea):
    for i in range(numero_maquinas):
        for j in asignacion_maquinas[i]:
            if tarea==j:
                return(i)

# Funcion mejor cambio de maquina

In [338]:
def mejor_cambio_maquina(tarea,tiempos_final_maquina):
    tiempos=[]
    for i in range(numero_maquinas):
        tiempos.append(tiempos_final_maquina[i]+matriz_tiempos[tarea][i])
    tiempos=np.asarray(tiempos)
    tiempo=np.min(tiempos)
    return(tiempo,np.argmin(tiempos))    

# Funcion para actualizar solucion

In [339]:
def actualizar_solucion(asignacion_maquinas,tiempo_maquina,tarea,maquina_vieja,maquina_nueva,matriz_tiempos):
    asignacion_maquinas[maquina_vieja].remove(tarea)
    asignacion_maquinas[maquina_nueva].append(tarea)
    tiempo_maquina[maquina_vieja].remove(matriz_tiempos[tarea][maquina_vieja])
    tiempo_maquina[maquina_nueva].append(matriz_tiempos[tarea][maquina_nueva])
    return(asignacion_maquinas,tiempo_maquina)

# Iterated Local Search

## Insertion local search

In [340]:
def insertion_local_search(pi,asignacion_maquinas,tiempos_final_maquina,tiempo_maquina):
    improved=True
    while improved:
        improved=False
        for j in range(numero_tareas):                
            i=maquina_por_tarea(j)
            l,maquina_nueva=mejor_cambio_maquina(j,tiempos_final_maquina)
            if l<tiempos_final_maquina[i]:
                improved=True
                asignacion_maquinas,tiempo_maquina=actualizar_solucion(asignacion_maquinas,tiempo_maquina,j,i \
                                                                       ,maquina_nueva,matriz_tiempos)
                pi,tiempos_final_maquina=evaluacion(numero_maquinas,tiempo_maquina)
    return(pi,tiempos_final_maquina,asignacion_maquinas,tiempo_maquina)
                

## Interchange Local Search

In [341]:
def interchange_local_search(pi_prima,tiempos_final_maquina,asignacion_maquinas,tiempo_maquina):
    pi=pi_prima
    improved=True
    while improved:
        improved=False
        for j_1 in range(numero_tareas):
            for j_2 in range(numero_tareas):
                i_1=maquina_por_tarea(j_1)
                i_2=maquina_por_tarea(j_2)
                if j_1!=j_2 and i_1!=i_2 and matriz_tiempos[j_2][i_1]+matriz_tiempos[j_1][i_2]< \
                matriz_tiempos[j_1][i_1]+matriz_tiempos[j_2][i_2] and (tiempos_final_maquina[i_1]+matriz_tiempos[j_2][i_1] \
                - matriz_tiempos[j_1][i_1])<c_max and (tiempos_final_maquina[i_2]+matriz_tiempos[j_1][i_2] \
                - matriz_tiempos[j_2][i_2])<c_max:
                    asignacion_maquinas,tiempo_maquina=actualizar_solucion(asignacion_maquinas,tiempo_maquina,j_1,i_1 \
                                                                           ,i_2,matriz_tiempos)
                    asignacion_maquinas,tiempo_maquina=actualizar_solucion(asignacion_maquinas,tiempo_maquina,j_2,i_2 \
                                                                           ,i_1,matriz_tiempos) 
                    pi,tiempos_final_maquina=evaluacion(numero_maquinas,tiempo_maquina)
                    improved=True
    return(pi,tiempos_final_maquina,asignacion_maquinas,tiempo_maquina)               

## Solution Modification

In [ ]:
def solution_modification():
    

## Main

In [342]:
pi=makespan
pi_best=pi
c_max=pi
tiempo_ejecucion=3
start=time.time()
end=0
fin=end-start
while fin <= tiempo_ejecucion: 
    improved=True
    while improved:
        improved=False
        pi_prima,tiempos_final_maquina,asignacion_maquinas,tiempo_maquina=insertion_local_search(pi,asignacion_maquinas,tiempos_final_maquina,tiempo_maquina)
        c_max=pi_prima
        print(pi_prima)
        pi_dosprima,tiempos_final_maquina,asignacion_maquinas,tiempo_maquina=interchange_local_search(pi_prima,tiempos_final_maquina,asignacion_maquinas,tiempo_maquina)
        print(pi_dosprima)
        if pi_dosprima!=pi_prima:
            improved=True
            pi=pi_dosprima
    if pi_dosprima<=pi_best:
        pi_best=pi_dosprima
        pi=solution_modification(pi_best)
    end=time.time()
    fin=end-start


140
135


SystemExit: 

C:\Users\ggalv\Anaconda2\envs\py35\lib\site-packages\IPython\core\interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
